## 학과 정보 수집

In [16]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame

In [ ]:
#접속객체 session
session = requests.Session()

session.headers.update({
    "Referer" : "",
    "user-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
})

## 응답결과로부터 데이터 추출

In [18]:
# 빈리스트에 추출한 데이터 저장
학과목록_리스트 = []

# 대학번호 리스트 정의 --> URL을 분석해서 얻은 값
clist = [1011,1023,1030,1041,1046,1052,1054,1057,1062,1064,1070,1073,1076]
for c in clist:
    url = "https://www.sookmyung.ac.kr/sookmyungkr/%d/subview.do" % c


    # 접속객체를 통해 api 접속
    r = session.get(url)

    #접속 실패시 오류메세지 출력
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" %(r.status_code, r.reason)
        raise Exception(msg)

    r.encoding = 'utf-8'
    soup = BeautifulSoup(r.text)
    soup


    college_list = soup.select(".college_list")
    print(college_list)

    for item in college_list:
        # 학과이름
        h5El = item.select("h5")
        name = h5El[0].text.strip()
        print(name)

        #상세보기 url
        viewEl = item.select(".view")
        view = viewEl[0].attrs['href']
        #만약 상세보기에 https://www.sookmyung.ac.kr이 없다면
        if view.find("https://www.sookmyung.ac.kr") == -1:
            view = "https://www.sookmyung.ac.kr" + view
        # print(view)

        #학과소개 pdf
        pdfEl = item.select(".info")
        pdf = pdfEl[0].attrs['href']
        # print(pdf)

        #학과홈페이지
        homepageEl = item.select(".homepage")
        homepage = homepageEl[0].attrs['href']
        # print(homepage)

        # 추출한 데이터를 딕셔너리에 묶고 학과목록_리스트에 append
        college_dict = {"학과이름" : name, "상세페이지" : view, "홈페이지": homepage}

# -----------------------------------------------------------------
# 상세보기 url 새롭게 수집
# ------------------------------------------------------------------
        r = session.get(view)

        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            print(msg)
            continue

        r.encoding = 'uft-8'
        detailSoup = BeautifulSoup(r.text)
        # print(detailSoup)

        # 전화번호, 팩스번호, 위치, 이메일주소 가져오기
        infoEl = detailSoup.select(".college_info_data dl")
        # print(infoEl)

        for info in infoEl:
            dt = info.select("dt")[0].text.replace(":","").strip()
            dd = info.select("dd")[0].text.replace(":","").strip()
            # print(dt,dd)
            college_dict[dt] = dd
# ----------------------------------------------------------------
# 상세보기 url 새롭게 수집 끝
# -----------------------------------------------------------------

    학과목록_리스트.append(college_dict)
    r = session.get(url, stream = True)
    if r.status_code != 200:
        r.encoding = 'uft-8'
        with open("%s.pdf" % name, 'wb') as f:
            f.write(r.raw.read())

# #학과목록_리스트를 데이터프레임으로 만들기
df = DataFrame(학과목록_리스트)

# #df파일을 엑셀형식으로 result.xlsx이름으로 저장
df.to_excel("result.xlsx")

df

[<div class="college_list">
<h5>화공생명공학부</h5>
<p class="img"><img alt="화공생명공학부 마스코트" src="/sites/sookmyungkr/images/sub/contents/college_mascot_0301.jpg"/></p>
<a class="view" href="/sookmyungkr/1031/subview.do">상세보기</a>
<a class="info" href="https://admission.sookmyung.ac.kr/admission/pdf/major/Sookmyung_Major_17_2022.pdf" target="_blank">학과소개</a>
<a class="homepage" href="http://cbe.sookmyung.ac.kr/" target="_blank">홈페이지</a>
</div>, <div class="college_list">
<h5 class="sub">인공지능공학부</h5>
<p class="img"><img alt="인공지능공학부 마스코트" src="/sites/sookmyungkr/images/sub/contents/college_mascot_0302.jpg"/></p>
<a class="view" href="/sookmyungkr/1033/subview.do">상세보기</a>
<a class="info" href="https://admission.sookmyung.ac.kr/admission/pdf/major/Sookmyung_Major_18_2022.pdf" target="_blank">학과소개</a>
<a class="homepage" href="http://it.sookmyung.ac.kr/" target="_blank">홈페이지</a>
</div>, <div class="college_list">
<h5 class="sub"><span>첨단소재·전자융합공학부</span>지능형전자시스템전공</h5>
<p class="img"><img alt="지능형전자

,학과이름,상세페이지,홈페이지,전화,위치,이메일
0,기초공학부,https://www.sookmyung.ac.kr/sookmyungkr/1040/s...,http://basic.sookmyung.ac.kr/,02-2077-7855,르네상스플라자 501,basic@sookmyung.ac.kr


#### find () 함수 : 문자열 추출 시 
text = "hello world, hello python"

target ="hello"
p = len(target)

t = 0

while t > -1:
    t = text.find("hello") #0                   
    if t > -1:
        text = text[t+p:] # world, hello python


### find()
#### 만약 찾을 문자가 존재하지 않는다면 -1반환. 
#### str.find('찾을 문자')
#### str.find('찾을 문자', 시작index)
#### str.find('찾을 문자', 시작index, 끝 index)